In [42]:
import pandas as pd
import glob
import os
from scipy.stats import zscore
from utils import find_nulls, add_ratio_features, add_z_score

In [43]:
DATASET_SUFFIX = '100'
PATH_SAMPLE = f'../dataset/real-time-{DATASET_SUFFIX}/train_preprocessed/realtime_bacon.csv'
CSV_BASE_FOLDER = f'../dataset/real-time-{DATASET_SUFFIX}/train_preprocessed'
CSV_PREPROCESSED_FOLDER = f'../dataset/real-time-{DATASET_SUFFIX}/train_preprocessed'

Z_SCORE_COLS =  ['trace_involved_amt','contract_block_involved','contract_tx_count','contract_main_active_days',
                'sender_block_involved','sender_tx_count','sender_main_active_days','contract_interact',
                'sender_tx_count_call_contract','sender_days_call_contract','trace_amt','distinct_sender_in_contract',
                'contract_lifetime_days','contract_lifetime_block','distinct_contract_sender_called',
                'sender_lifetime_days','sender_lifetime_block','contract_involved_amt','max_breadth','depth',
                'distinct_was_called_in_sample','distinct_sender_call_in_sample','gas','gas_price',
                'receipt_cumulative_gas_used','receipt_gas_used','value','nonce']

SUS_FILE = '../dataset/sus_tx.csv'  # Replace with the actual file path

In [44]:
# Check if the directory exists
if not os.path.exists(CSV_PREPROCESSED_FOLDER):
    # Create the directory if it doesn't exist
    os.makedirs(CSV_PREPROCESSED_FOLDER)
    print(f"Directory '{CSV_PREPROCESSED_FOLDER}' created successfully.")
else:
    print(f"Directory '{CSV_PREPROCESSED_FOLDER}' already exists.")

Directory '../dataset/real-time-100/train_preprocessed' already exists.


In [45]:
def impute_depth_and_max_breadth(path):
    df = pd.read_csv(path)
    df['max_breadth'] = df['max_breadth']+1
    df['max_breadth'] = df['max_breadth'].fillna(0)
    df['depth'] = df['depth'].fillna(0)
    return df

def add_sus_col(df,attack_transactions):

    def set_value_based_on_condition(row):
        if (row['transaction_hash'] in attack_transactions):
            return 1
        else:
            return 0
        
    df['is_sus'] = df.apply(set_value_based_on_condition, axis=1)

    sus_data = df[df['is_sus'] == 1]
    not_sus_data = df[df['is_sus'] == 0]

    print(len(sus_data))
    print(len(not_sus_data))

    return df

In [46]:
sample_df = pd.read_csv(PATH_SAMPLE)
sample_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 76 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   index                                   100 non-null    int64  
 1   transaction_hash                        100 non-null    object 
 2   trace_involved_amt                      100 non-null    int64  
 3   from_address                            100 non-null    object 
 4   to_address                              100 non-null    object 
 5   block_timestamp                         100 non-null    object 
 6   contract_block_involved                 100 non-null    int64  
 7   contract_tx_count                       100 non-null    int64  
 8   contract_block_per_tx                   100 non-null    float64
 9   contract_main_active_days               100 non-null    int64  
 10  sender_block_involved                   100 non-null    int64  

In [47]:
for file_path in glob.glob(os.path.join(CSV_BASE_FOLDER, "*.csv")):
    df = pd.read_csv(file_path)
    find_nulls(df)

Series([], dtype: int64)
Series([], dtype: int64)
Series([], dtype: int64)
Series([], dtype: int64)
Series([], dtype: int64)
Series([], dtype: int64)
Series([], dtype: int64)
Series([], dtype: int64)
Series([], dtype: int64)
Series([], dtype: int64)
Series([], dtype: int64)
Series([], dtype: int64)
Series([], dtype: int64)


In [48]:
for file_path in glob.glob(os.path.join(CSV_BASE_FOLDER, "*.csv")):
    filename = os.path.basename(file_path)
    print(filename)
    preprocessed_file_path = CSV_PREPROCESSED_FOLDER+'/'+filename
    imputed_df = impute_depth_and_max_breadth(file_path)

    imputed_df['value'] = imputed_df['value'].astype('float32')

    z_score_df = add_z_score(imputed_df,Z_SCORE_COLS)

    ratio_df = add_ratio_features(z_score_df)

    sus_df = pd.read_csv(SUS_FILE)
    sus_transactions = sus_df['sus_tx']
    sus_transactions = set(sus_transactions)
    added_sus_df = add_sus_col(ratio_df,sus_transactions)

    print('Preprocessed file:',filename)
    find_nulls(z_score_df)

    added_sus_df.to_csv(preprocessed_file_path,index=False)
    

realtime_bacon.csv


1
99
Preprocessed file: realtime_bacon.csv
Series([], dtype: int64)
realtime_cream.csv
11
89
Preprocessed file: realtime_cream.csv
Series([], dtype: int64)
realtime_dfx.csv


c:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\core\_methods.py:176: RuntimeWarning: overflow encountered in multiply
  x = um.multiply(x, x, out=x)
c:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\core\_methods.py:187: RuntimeWarning: overflow encountered in reduce
  ret = umr_sum(x, axis, dtype, out, keepdims=keepdims, where=where)


6
49
Preprocessed file: realtime_dfx.csv
Series([], dtype: int64)
realtime_fei.csv
1
99
Preprocessed file: realtime_fei.csv
Series([], dtype: int64)
realtime_hypebear.csv
1
99
Preprocessed file: realtime_hypebear.csv
Series([], dtype: int64)


c:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\core\_methods.py:176: RuntimeWarning: overflow encountered in multiply
  x = um.multiply(x, x, out=x)
c:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\core\_methods.py:187: RuntimeWarning: overflow encountered in reduce
  ret = umr_sum(x, axis, dtype, out, keepdims=keepdims, where=where)
c:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\core\_methods.py:176: RuntimeWarning: overflow encountered in multiply
  x = um.multiply(x, x, out=x)


realtime_jay.csv
1
99
Preprocessed file: realtime_jay.csv
Series([], dtype: int64)
realtime_noodle.csv
1
99
Preprocessed file: realtime_noodle.csv
Series([], dtype: int64)
realtime_omni.csv
3
97
Preprocessed file: realtime_omni.csv
Series([], dtype: int64)
realtime_orion.csv


c:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\core\_methods.py:176: RuntimeWarning: overflow encountered in multiply
  x = um.multiply(x, x, out=x)
c:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\core\_methods.py:187: RuntimeWarning: overflow encountered in reduce
  ret = umr_sum(x, axis, dtype, out, keepdims=keepdims, where=where)
c:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\core\_methods.py:176: RuntimeWarning: overflow encountered in multiply
  x = um.multiply(x, x, out=x)
c:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\core\_methods.py:176: RuntimeWarning: overflow encountered in multiply
  x = um.multiply(x, x, out=x)


1
99
Preprocessed file: realtime_orion.csv
Series([], dtype: int64)
realtime_rari.csv
49
51
Preprocessed file: realtime_rari.csv
Series([], dtype: int64)
realtime_revest.csv
2
98
Preprocessed file: realtime_revest.csv
Series([], dtype: int64)
realtime_sanshu.csv
3
97
Preprocessed file: realtime_sanshu.csv
Series([], dtype: int64)
realtime_visor.csv
1
99
Preprocessed file: realtime_visor.csv
Series([], dtype: int64)


c:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\core\_methods.py:176: RuntimeWarning: overflow encountered in multiply
  x = um.multiply(x, x, out=x)
